## This is a notebook that verify the communication of the pointing kernel with the Mount
It is part of a series of notebook that tests the communication between the pointing kernel and different components:
- the mount
- the rotator
- M1M3
- M2
- Cam hexapod **
- M2 hexapod **

We will leave ComCam out of this notebook

The goals of the exercice are 
- Check heartbeat from all components.
- Make sure we can put all components in ENABLE state and back to STANDBY.
- Check the EFD to ensure that the correct events/commands/telemetry is published

This will ensure that we are ready to check the slew when needed


In [ ]:
#from lsst.ts.observatory.maintel import MTCS
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

import asyncio

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime


In [ ]:
#domain = salobj.Domain()
# I don't need this if I am calling a controller using script

use NewMTMount to receive and send telemetry. This is associated with the CSC (and not the code that Tekniker wrote)

In [ ]:
script = salobj.Controller("Script", index=1)
mtptg = salobj.Remote(script.domain, "MTPtg")
mtm = salobj.Remote(script.domain, "NewMTMount")
camHex = salobj.Remote(script.domain, "Hexapod", index = '1')
m2Hex = salobj.Remote(script.domain, "Hexapod", index = '2')
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await asyncio.gather(mtptg.start_task,
                     script.start_task,
                     mtm.start_task,
                    camHex.start_task,
                    m2Hex.start_task)

In [ ]:
await mtm.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await m2Hex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
#Check which hexapod we are controlling
whichHex1 = await camHex.configuration.aget(timeout=10.)
print("Hexapod camHex", whichHex1.HexapodID)
print(camHex.salinfo.index)

whichHex2 = await m2Hex.configuration.aget(timeout=10.)
print("Hexapod m2Hex", whichHex2.HexapodID)
print(m2Hex.salinfo.index)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(mtm, salobj.State.ENABLED)
await camHex.set_summary_state(camHex, salobj.State.ENABLED)
await m2Hex.set_summary_state(m2Hex, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
now = datetime.now()
print("Start to point the telescope", now)

alt = 75. * u.deg
az = 25. * u.deg
rot_tel = Angle(20, unit= u.deg) 

target_name="TMA motion test"
#getting time for the pointing to ensure that we're using the time that the mount seems it is
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
#Convert the astropy class
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
#Check that we're getting the right time
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")
print(curr_time_mtptg.tai.value)

#create the object with the alt az coordinate + time to be able to convert them in RA and Dec
cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_mtptg.tai, 
                location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters      
rot_pa = rot_tel
print(rot_pa)  


In [ ]:
#The pointing component is commanding the mount directly
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

Check Commands... apparently we need to check the telemetry on the mtmount instead of newmtmount

In [ ]:
mtmold = salobj.Remote(script.domain, "MTMount")
rot = salobj.Remote(script.domain, "Rotator")

In [ ]:
print(mtm.salinfo.command_names)

Check Telemetry

In [ ]:
camHexPosition = await camHex.actuators.aget(timeout=10.)
print("camera Hexapod positions", camHexPosition.HexapodID,  camHexPosition.calibrated0,  camHexPosition.calibrated1,  camHexPosition.calibrated2,  camHexPosition.calibrated3,  camHexPosition.calibrated4,  camHexPosition.calibrated5)

In [ ]:
m2HexPosition = await m2Hex.actuators.aget(timeout=10.)
print("M2 Hexapod positions", m2HexPosition.HexapodID,  m2HexPosition.calibrated0,  m2HexPosition.calibrated1,  m2HexPosition.calibrated2,  m2HexPosition.calibrated3,  m2HexPosition.calibrated4,  m2HexPosition.calibrated5)

Check Events

In [ ]:
print(mtm.salinfo.event_names)

In [ ]:
target = await mtm.evt_target.aget() 
#mtm.tel_Elevation.aget()
rotAngle = await rot.evt_target.aget()

In [ ]:
now = datetime.now()
print(now)
print("elevation",target.elevation)
print("Azimuth",target.azimuth)
print("Rotator",rotAngle.position)

I now want to do the same but with a field coordinate instead of a Alt Az mount position

#what is the official way to shutdown the system.

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)